In [1]:
import re

import boto3


class Coder:
    def __init__(self):
        # self.model_id = "anthropic.claude-3-haiku-20240307-v1:0"
        self.model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
        self.bedrock_client = boto3.client(
            service_name="bedrock-runtime",
            region_name="us-east-1",
        )
        self.base_prompts = """
You are a professional programmer. Please fill one line code according to <prefix> and <suffix>. Your infill code should be in <middle>.

Here is an example:

Human:
<prefix>def add_two_numbers(a, b):
    result = (</prefix>
<suffix>)
    return result</suffix>


Assistant:
<middle> a + b</middle>


Here is the prefix and suffix of the code you need to infill:

Human:
<prefix>{{PREFIX}}</prefix>
<suffix>{{SUFFIX}}</suffix>
"""
        self.prefill = "<middle>"

    def generate(self, prefix, suffix, middle=""):
        prompts = self.base_prompts.replace("{{PREFIX}}", prefix).replace(
            "{{SUFFIX}}", suffix
        )
        messages = [
            {
                "role": "user",
                "content": [
                    {"text": prompts},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"text": self.prefill + middle},
                ],
            },
        ]

        response = self.bedrock_client.converse(
            modelId=self.model_id,
            messages=messages,
            inferenceConfig={
                "temperature": 0.0,
                "topP": 1,
            },
            additionalModelRequestFields={"top_k": 1},
        )

        ret = (
            self.prefill + middle + response["output"]["message"]["content"][0]["text"]
        )
        ret = re.search(r"<middle>(.*)</middle>", ret, re.DOTALL).group(1)
        return ret


In [2]:
code = Coder()

prefix = """
@Data
public class Response {
 private String model;
 private Date created_at;
 private String response;
 @JsonProperty(value = "eval_count")
 private Integer eval_count;
"""

suffix = """)
 private Long eval_duration;
}
"""

middle = """ @JsonProperty("""
middle = code.generate(prefix, suffix, middle)
print(middle)
print()
print(prefix + middle + suffix)


 @JsonProperty(value = "eval_duration"


@Data
public class Response {
 private String model;
 private Date created_at;
 private String response;
 @JsonProperty(value = "eval_count")
 private Integer eval_count;
 @JsonProperty(value = "eval_duration")
 private Long eval_duration;
}

